In [62]:
import pandas as pd

# Phase 1: Data Discovery

In [63]:
df = pd.read_csv("springfield_azure_resources.csv")
columns = ['resource_name', 'resource_type', 'location', 'owner', 'environment', 'subscription_id', 'resource_group_name', 'resource_id', 'sku','migration_wave']

df = df[columns]

In [64]:
df.head()

,resource_name,resource_type,location,owner,environment,subscription_id,resource_group_name,resource_id,sku,migration_wave
0,sa-dev-purpose-251,Microsoft.Storage/storageAccounts,East US,Lisa,dev,c3d4e5f6-g7h8-9012-cdef-123456789012,Lisa-dev-rg,/subscriptions/c3d4e5f6-g7h8-9012-cdef-1234567...,Standard_LRS,wave1
1,robinsonplc-sql-95,Microsoft.Sql/servers,eastus,Milhouse,staging,b2c3d4e5-f6g7-8901-bcde-f12345678901,springfield-staging-resources,/subscriptions/b2c3d4e5-f6g7-8901-bcde-f123456...,Premium,WAVE 2
2,springfield-sa-rate-staging,Microsoft.Storage/storageAccounts,westus,Ned,staging,b2c3d4e5-f6g7-8901-bcde-f12345678901,RG-STAGING-NED,/subscriptions/b2c3d4e5-f6g7-8901-bcde-f123456...,Premium_LRS,WAVE 2
3,springfield-vnet-save-prod,Microsoft.Network/virtualNetworks,Springfield,Homer,prod,b2c3d4e5-f6g7-8901-bcde-f12345678901,Homer-prod-rg,/subscriptions/b2c3d4e5-f6g7-8901-bcde-f123456...,Basic,Wave 3
4,vm-prod-husband-164,Microsoft.Compute/virtualMachines,east-us,Homer,prod,c3d4e5f6-g7h8-9012-cdef-123456789012,Homer-prod-rg,/subscriptions/c3d4e5f6-g7h8-9012-cdef-1234567...,STANDARD_D2S_V3,Wave 2


In [65]:
df.describe()

,resource_name,resource_type,location,owner,environment,subscription_id,resource_group_name,resource_id,sku,migration_wave
count,2500,2500,2500,2500,2500,2500,2500,2500,2500,2500
unique,2497,10,9,7,12,3,716,2500,16,6
top,springfield-nsg-radio-STAGING,Microsoft.Network/networkSecurityGroups,westus,Marge,test,b2c3d4e5-f6g7-8901-bcde-f12345678901,springfield-STAGING-resources,/subscriptions/b2c3d4e5-f6g7-8901-bcde-f123456...,Premium,wave1
freq,2,294,314,715,235,840,48,1,660,453


# Phase 2: Data Cleansing

## Evaluate Owners

In [66]:
df['owner'].value_counts()

owner
Marge       715
Homer       651
Lisa        529
Bart        213
Ned         148
Milhouse    132
Apu         112
Name: count, dtype: int64

## Check environments

In [67]:
df['environment'].value_counts()

environment
test           235
Dev            230
STAGING        221
staging        212
uat            212
production     211
dev            210
prod           204
PROD           196
qa             195
Test           195
development    179
Name: count, dtype: int64

In [68]:
df['environment'] = (
    df['environment']
    .apply(lambda x: x.lower().strip()[:3])
    .apply(lambda x: {'pro':'prod', 'tes':'test', 'sta':'staging'}.get(x, x))
)

df = df.loc[df['environment'].isin(['dev', 'prod', 'test'])]


df['environment'].value_counts()

environment
dev     619
prod    611
test    430
Name: count, dtype: int64

## Check Locations

In [69]:
df['location'].value_counts()

location
westus         213
eastus         193
east-us        189
Springfield    188
uksouth        187
uk-south       184
East US        176
Shelbyville    165
UK South       165
Name: count, dtype: int64

In [70]:
df['location'] = df['location'].apply(lambda x: x.lower().strip().replace(" ", "").replace("-", ""))
df['location'].value_counts()

location
eastus         558
uksouth        536
westus         213
springfield    188
shelbyville    165
Name: count, dtype: int64

## Wrangle migration wave

In [71]:
df['migration_wave'].value_counts()

migration_wave
wave1     312
Wave 1    293
WAVE 2    282
Wave 2    275
Wave 4    268
Wave 3    230
Name: count, dtype: int64

In [72]:
df['migration_wave'] = df['migration_wave'].apply(lambda x: x.lower().replace(" ", "").replace("wave","")).astype(int)
df['migration_wave'].value_counts()

migration_wave
1    605
2    557
4    268
3    230
Name: count, dtype: int64

## Business Rule 1: All locations must be 'uksouth'

In [73]:
df['location'].value_counts()

location
eastus         558
uksouth        536
westus         213
springfield    188
shelbyville    165
Name: count, dtype: int64

In [74]:
df['location'] = 'uksouth'

In [75]:
df['location'].value_counts()

location
uksouth    1660
Name: count, dtype: int64

## Business Rule 2:

### Data Discovery

In [76]:
pivot = (
    df
    .groupby(['environment','owner'])
    .size()
    .unstack(fill_value=0)               # owners → columns, fill missing with zero
)

print(pivot)


owner        Apu  Bart  Homer  Lisa  Marge  Milhouse  Ned
environment                                              
dev           27    50    169   118    184        27   44
prod          24    49    170   130    169        33   36
test          16    35    115    90    116        27   31


In [77]:
df.groupby(['environment','owner']).size().index

MultiIndex([( 'dev',      'Apu'),
            ( 'dev',     'Bart'),
            ( 'dev',    'Homer'),
            ( 'dev',     'Lisa'),
            ( 'dev',    'Marge'),
            ( 'dev', 'Milhouse'),
            ( 'dev',      'Ned'),
            ('prod',      'Apu'),
            ('prod',     'Bart'),
            ('prod',    'Homer'),
            ('prod',     'Lisa'),
            ('prod',    'Marge'),
            ('prod', 'Milhouse'),
            ('prod',      'Ned'),
            ('test',      'Apu'),
            ('test',     'Bart'),
            ('test',    'Homer'),
            ('test',     'Lisa'),
            ('test',    'Marge'),
            ('test', 'Milhouse'),
            ('test',      'Ned')],
           names=['environment', 'owner'])

In [78]:
df.groupby(['environment','owner']).size().unstack().index

Index(['dev', 'prod', 'test'], dtype='object', name='environment')

In [79]:
df.groupby(['environment','owner']).size().unstack().columns

Index(['Apu', 'Bart', 'Homer', 'Lisa', 'Marge', 'Milhouse', 'Ned'], dtype='object', name='owner')

### Implementation

In [80]:
df['owner'] = df['owner'].apply(lambda x: {'Apu':'Marge', 'Milhouse':'Lisa', 'Ned':'Homer', 'Bart':'Homer'}.get(x, x))

In [81]:
pivot = (
    df
    .groupby(['environment','owner'])
    .size()
    .unstack(fill_value=0)               # owners → columns, fill missing with zero
)

print(pivot)


owner        Homer  Lisa  Marge
environment                    
dev            263   145    211
prod           255   163    193
test           181   117    132


In [82]:
df.head()

,resource_name,resource_type,location,owner,environment,subscription_id,resource_group_name,resource_id,sku,migration_wave
0,sa-dev-purpose-251,Microsoft.Storage/storageAccounts,uksouth,Lisa,dev,c3d4e5f6-g7h8-9012-cdef-123456789012,Lisa-dev-rg,/subscriptions/c3d4e5f6-g7h8-9012-cdef-1234567...,Standard_LRS,1
3,springfield-vnet-save-prod,Microsoft.Network/virtualNetworks,uksouth,Homer,prod,b2c3d4e5-f6g7-8901-bcde-f12345678901,Homer-prod-rg,/subscriptions/b2c3d4e5-f6g7-8901-bcde-f123456...,Basic,3
4,vm-prod-husband-164,Microsoft.Compute/virtualMachines,uksouth,Homer,prod,c3d4e5f6-g7h8-9012-cdef-123456789012,Homer-prod-rg,/subscriptions/c3d4e5f6-g7h8-9012-cdef-1234567...,STANDARD_D2S_V3,2
6,kvdevelopmentspring78,Microsoft.KeyVault/vaults,uksouth,Homer,dev,c3d4e5f6-g7h8-9012-cdef-123456789012,RG-DEVELOPMENT-HOMER,/subscriptions/c3d4e5f6-g7h8-9012-cdef-1234567...,Basic,1
7,app-dev-prove-647,Microsoft.Web/sites,uksouth,Homer,dev,a1b2c3d4-e5f6-7890-abcd-ef1234567890,springfield-dev-resources,/subscriptions/a1b2c3d4-e5f6-7890-abcd-ef12345...,Standard,1


In [83]:
from azure.identity import DefaultAzureCredential

In [84]:
credential = DefaultAzureCredential()


In [85]:
credential

In [86]:
from azure.mgmt.resource import ResourceManagementClient

In [87]:
client = ResourceManagementClient(credential, "springfield-sub-12345")


In [88]:
dev_df = df.iloc[0:20]

In [89]:
# Create resource groups first
resource_groups = dev_df['resource_group_name'].unique()
for rg_name in resource_groups:
    try:
        client.resource_groups.create_or_update(
            rg_name,
            {"location": "uksouth", "tags": {"migration": "springfield"}}
        )
        print(f"✓ Created resource group: {rg_name}")
    except Exception as e:
        print(f"✗ Failed to create {rg_name}: {e}")

✓ Created resource group: Lisa-dev-rg
✓ Created resource group: Homer-prod-rg
✓ Created resource group: RG-DEVELOPMENT-HOMER
✓ Created resource group: springfield-dev-resources
✓ Created resource group: RG-DEV-MARGE
✓ Created resource group: RG-TEST-HOMER
✓ Created resource group: resourcegroup-dev-nothing
✓ Created resource group: resourcegroup-Dev-Mr
✓ Created resource group: RG-PRODUCTION-LISA
✓ Created resource group: Bart-test-rg
✓ Created resource group: Milhouse-production-rg
✓ Created resource group: Marge-development-rg
✓ Created resource group: springfield-Test-resources
✓ Created resource group: RG-TEST-LISA
✓ Created resource group: rg-PROD-marge
✓ Created resource group: rg-prod-marge
✓ Created resource group: Lisa-development-rg


In [93]:
# Prepare resources for bulk creation
resources_data = []
for _, row in df.iterrows():
    resources_data.append({
        'name': row['resource_name'],
        'resource_type': row['resource_type'],
        'resource_group': row['resource_group_name'],
        'location': 'uksouth',  # Standardized location
        'tags': {
            'owner': row['owner'],
            'environment': row['environment'],
            'migration_wave': str(row['migration_wave'])
        }
    })

# Bulk create resources
print(f"\nCreating {len(resources_data)} resources...")

def progress_callback(tracker):
    print(f"\rProgress: {tracker.percentage:.1f}% "
            f"({tracker.completed} completed, {tracker.failed} failed)", 
            end='', flush=True)

tracker = WorkshopUtilities.bulk_create_resources(
    client, 
    resources_data, 
    progress_callback
)

print(f"\n\nMigration complete!")
print(f"Total time: {tracker.elapsed_time:.2f} seconds")
print(f"Success rate: {(tracker.completed/tracker.total)*100:.1f}%")


Creating 1660 resources...


NameError: name 'WorkshopUtilities' is not defined